In [3]:
import pandas as pd
import glob
import numpy as np
import dapla as dp

In [1]:
#Sjekker hva som ligger i mappen:

paths =  spark.read.path('/produkt/kostra/regnskap/demo/utpakket/*')
paths.show(100, False)

#paths = glob.glob("/produkt/kostra/regnskap/demo/utpakket/*")
#paths

#På linux (prod) blir dette:
#paths = glob.glob("/ssb/stamme03/komakro/komm_aarsregn_ubearb/arkiv/primk_regnskap/*")
#paths

+-------------------------------------------------------------------------------------+
|path                                                                                 |
+-------------------------------------------------------------------------------------+
|/produkt/kostra/regnskap/demo/utpakket/ds_20210209                                   |
|/produkt/kostra/regnskap/demo/utpakket/ds_artshierarki_20210209                      |
|/produkt/kostra/regnskap/demo/utpakket/ds_artshierarki_nettinger_kasse_20210210      |
|/produkt/kostra/regnskap/demo/utpakket/ds_artshierarki_nettinger_saerbedrift_20210210|
|/produkt/kostra/regnskap/demo/utpakket/ds_formler_20210210                           |
|/produkt/kostra/regnskap/demo/utpakket/ds_funksjonshierarki_20210210                 |
|/produkt/kostra/regnskap/demo/utpakket/ds_kombinasjoner_20210210                     |
|/produkt/kostra/regnskap/demo/utpakket/ds_saerbedrift_20210210                       |
+-------------------------------

In [6]:
regn = dp.read_pandas('/produkt/kostra/regnskap/demo/utpakket/ds_20210209')
regn.info()
regn.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683692 entries, 0 to 683691
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   art              683692 non-null  object
 1   funksjon         683692 non-null  object
 2   fylkesregion     683692 non-null  object
 3   kontoklasse      683692 non-null  object
 4   periode          683692 non-null  object
 5   regnskapsomfang  683692 non-null  object
 6   belop            683692 non-null  int32 
dtypes: int32(1), object(6)
memory usage: 33.9+ MB


,art,funksjon,fylkesregion,kontoklasse,periode,regnskapsomfang,belop
0,010,400,0100,1,2019,B,942
1,240,400,0100,1,2019,B,58
2,050,400,0100,1,2019,B,87
3,080,400,0100,1,2019,B,17136
4,090,400,0100,1,2019,B,2105


In [ ]:
#Henter filen fra linux, sjekker innhold
#regn_2020 = pd.read_table("/ssb/stamme03/komakro/komm_aarsregn_ubearb/arkiv/primk_regnskap/g2020.txt")

In [ ]:
#Fil med faste posisjoner. Må benytte fwf (fixed width) og definere posisjonene
#regn_2020 = pd.read_fwf("/ssb/stamme03/komakro/komm_aarsregn_ubearb/arkiv/primk_regnskap/g2020.txt",
#    names=["skjema", "periode", "kvartal", "komnr", "orgnr", "foretaksnr" ,"kontoklasse", "funksjon", "art", "belop"],
#    dtype=object,
#    na_values={"."," ."},
#    widths=[2, 4, 1, 6, 9, 9, 1, 4, 3, 9]
#)
#regn_2020.info()
#regn_2020.head(2)

In [7]:
len(regn)

683692

In [8]:
regn.dtypes

art                object
funksjon           object
fylkesregion       object
kontoklasse        object
periode            object
regnskapsomfang    object
belop               int32
dtype: object

In [10]:
#Retter datatyde om nødvendig
#regn_2020['belop'] = regn_2020['belop'].astype("int")
#regn_2020['kvartal'] = regn_2020['kvartal'].astype("str")
#regn_2020['orgnr'] = regn_2020['orgnr'].astype("str")
#regn_2020['foretaksnr'] = regn_2020['foretaksnr'].astype("str")
regn.convert_dtypes().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683692 entries, 0 to 683691
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   art              683692 non-null  string
 1   funksjon         683692 non-null  string
 2   fylkesregion     683692 non-null  string
 3   kontoklasse      683692 non-null  string
 4   periode          683692 non-null  string
 5   regnskapsomfang  683692 non-null  string
 6   belop            683692 non-null  Int32 
dtypes: Int32(1), string(6)
memory usage: 34.6 MB


In [11]:
#Lagrer til Parquet
regn.to_parquet("../../testdata/regnskap_0A_2019.parquet")

FileNotFoundError: [Errno 2] No such file or directory: '../../testdata/regnskap_0A_2019.parquet'

In [ ]:
#Henter inn parquet-fil
regn = pd.read_parquet("../../data/arbeid_mtk.parquet")

In [ ]:
regn.info()

In [ ]:
#Ajekker om det er duplikater
regn.duplicated()
regn[regn.duplicated()]

In [ ]:
#Filtrerer ut Oslo
regn[regn['komnr'] == '0301']

In [ ]:
#Filtrerer ut Oslo og driftsregnskapet
regn[(regn['komnr'].str[:2] == '03') & (regn['kontoklasse'] == '1')]

In [ ]:
Antall unike verdier i komnr
regn['komnr'].value_counts()

In [ ]:
Kjører en spørring 
x = regn.query("funksjon == '234' & art == '185'")
x

In [ ]:
#Sjekker om de to første posisjonene i komnr er siffer
regn["komnr"].astype(str).str[2].str.isdigit()

In [ ]:
#Filtrerer i regnskapsfilen
regn[(regn["funksjon"] == "234") & (regn["art"] == "185")]

In [ ]:
#Lager et objekt med spesifikasjoner for å filtrere i hovedfilen
maske = (regn["kontoklasse"] == "1") & (regn["funksjon"] == "234") & (regn["art"] == "185")
regn[maske]